In [12]:
from data_handler import return_create_statement_from_dataframe
import pandas as pd
from db_handler import create_connection,execute_query, close_connection
from data_handler import insert_statements_from_dataframe,extract_data_into_df,create_etl_watermark_table,insert_data_in_batches
from data_handler import execute_sql_script_from_config
from lookups import FileType
import prehook
from prehook import execute



db_session = create_connection()

In [2]:
prehook.execute()


V1_prehook_Create_ETL_watermark_table.sql
V1_prehook_Create_Schema.sql
V2_prehook_create_ETL_watermark_table.sql
V3_prehook_create_stg_kaggle_spotify_tracks.sql
V4_prehook_create_stg_kaggle_spotify_users.sql
_prehook_create_stg_kaggle_spotify_tracks.sql
_prehook_create_stg_kaggle_spotify_users.sql


UniqueViolation: duplicate key value violates unique constraint "stg_kaggle_spotify_users_pkey"
DETAIL:  Key (id)=(20000) already exists.


In [4]:
execute_sql_script_from_config('SQL_Commands/Create_genre_mapping_table.sql')
df = pd.read_excel("C:\\dataproject\\genre_mapping.xlsx")
insert_statement = insert_statements_from_dataframe(df, 'musicschema', 'genre_mapping')
# insert_statements2 = insert_statements_from_dataframe(df1, 'musicschema', 'stg_kaggle_spotify_tracks')

with db_session.cursor() as cursor:
    for statement in insert_statement:
        cursor.execute(statement)
    
db_session.commit()

SQL script executed successfully.


UniqueViolation: duplicate key value violates unique constraint "genre_mapping_pkey"
DETAIL:  Key (genre_id)=(1) already exists.


In [5]:
execute_sql_script_from_config('SQL_Commands/V1_prehook_Create_ETL_watermark_table.sql')

execute_sql_script_from_config('SQL_Commands/V2_hook_create_dim_users_table.sql')

execute_sql_script_from_config('SQL_Commands/V3_hook_create_dim_genre.sql')

execute_sql_script_from_config('SQL_Commands/V4_hook_create_dim_songs.sql')

execute_sql_script_from_config('SQL_Commands/V5_hook_create_fct_user_genre.sql') 


execute_sql_script_from_config('SQL_Commands/V9_hook_fct_average_listen_time_per_genre.sql')


SQL script executed successfully.
SQL script executed successfully.
SQL script executed successfully.
SQL script executed successfully.
SQL script executed successfully.
Error executing SQL script: function avg(text) does not exist
LINE 13:     AVG(music_time_slot) AS average_listen_time
             ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.



In [7]:


execute_sql_script_from_config('SQL_Commands/V8_hook_create_fct_age_distribution_per_genre.sql')

#execute_sql_script_from_config('SQL_Commands/V9_hook_create_fct_average_listen_time_per_genre.sql') #

execute_sql_script_from_config('SQL_Commands/V10_hook_create_fct_song_genre.sql') #

execute_sql_script_from_config('SQL_Commands/V11_hook_create_agg_songs_per_genre.sql') #fawte table genre mapping

execute_sql_script_from_config('SQL_Commands/V12_hook_create_agg_user_per_genre.sql') #

execute_sql_script_from_config('SQL_Commands/V13_hook_create_agg_genre.sql')

execute_sql_script_from_config('SQL_Commands/V14_hook_create_agg_release_date.sql')

execute_sql_script_from_config('SQL_Commands/V15_hook_create_agg_playlist.sql')

Error executing SQL script: relation "musicschema.fct_average_listen_time_per_genre" does not exist
LINE 7: INSERT INTO musicschema.fct_average_listen_time_per_genre (p...
                    ^

Error executing SQL script: column "id" does not exist
LINE 12:     id,
             ^
HINT:  There is a column named "id" in table "fct_song_genre", but it cannot be referenced from this part of the query.

Error executing SQL script: duplicate key value violates unique constraint "aggregate_songs_per_genre_pkey"
DETAIL:  Key (genre_id)=(4) already exists.

Error executing SQL script: duplicate key value violates unique constraint "aggregate_users_per_genre_pkey"
DETAIL:  Key (genre_id)=(4) already exists.

SQL script executed successfully.
SQL script executed successfully.
SQL script executed successfully.


<!DOCTYPE html><html lang="ar"><head><script nonce="6LK-jlbgAgjeDMVkJLQ3IQ">var DOCS_timing={}; DOCS_timing['pls']=new Date().getTime();</script><meta property="og:title" content="songs_dataset.xlsx"><meta property="og:type" content="article"><meta property="og:site_name" content="Google Docs"><meta property="og:url" content="https://docs.google.com/spreadsheets/d/1EDDNjMY23piFi8zCGaySGvlRcbUUqhKc/edit?usp=drive_link&amp;ouid=113060593632878646364&amp;rtpof=true&amp;sd=true&amp;usp=embed_facebook"><meta property="og:image" content="https://lh7-us.googleusercontent.com/docs/AHkbwyKCRFU4W70b5PQE-O4qf_I8CqrV3pNPPw3pma2FEQsBCgYWJ8HyuxCcfEhBvQz4ubIQvrVhPnnfAfT-2IEaoAcArC0l8JUV=w1200-h630-p"><meta property="og:image:width" content="1200"><meta property="og:image:height" content="630"><meta property="og:description" content="Sheet1

track_id,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,energy,key,loudness,mode,spee